# Aprendiendo AWS: AutoML con SageMaker Autopilot

Nota: este notebook debe ser corrido con el kernel `Python 3 (Data Science)` de Amazon SageMaker.

## Contenido

1. [Introducción](#Introduction)
2. [Pre-requisitos](#Pre-requisites)
3. [Entrenamiento del modelo con Autopilot](#Settingup)
4. [Evaluación del modelo: inferencias *online*](#OnlineInference)
5. [Evaluación del modelo: inferencias *batch*](#BatchInference)
6. [Eliminación/borrado de recursos](#Cleanup)

## 1. Introducción<a name="Introduction"></a>

Amazon SageMaker Autopilot es una solución que permite automatizar tareas de machine learning para juegos de datos en formato tabular (este tipo de soluciones también se conocen comúnmente como AutoML). En este notebook, vamos a usar los SDKs de AWS para acompañar el tutorial de video en el sitio de Aprendiendo AWS, y de esa manera generar un caso que te permita comprender el funcionamiento de SageMaker Autopilot: descargar los datos, crear el modelo, desplegarlo, y realizar inferencias sobre el mismo.

A lo largo de este ejemplo, vamos a usar un dataset para entrenar un modelo que nos permita predecir si un cliente determinado aceptará una propuesta del equipo de televentas para suscribir a un nuevo producto financiero (depósito o plazo fijo) que está siendo lanzado por el banco. Vamos a usar una colección de datos denominada [Bank Marketing Data Set](https://archive.ics.uci.edu/ml/datasets/bank+marketing), que nos permitirá modelar los datos que típicamente están disponibles en el momento de planificar y ejecutar este tipo de campañas. Para más información sobre el dataset, visitar este [enlace](https://archive.ics.uci.edu/ml/datasets/bank+marketing) .

Las campañas de marketing directo a través de correo, llamadas telefóncas, etc., son una táctica común que permite capturar nuevos clientes. Debido a que los recursos que posee el equipo de televentas son limitados, el objetivo de nuestro análisis será el de identificar un subconjunto de potenciales clientes que tengan la mayor posibilidad de aceptar la oferta ofrecida por el equipo de televentas. Para esto, vamos a entrenar un modelo que permita realizar la predicción a partir de información ya disponible en la organización, como ser por ejemplo información demográfica, interacciones anteriores e información de contexto.

Este *notebook* demuestra cómo puedes aplicar Autopilot y así producir un *pipeline* de procesamiento de machine learning para explorar una gran cantidad de potenciales modelos o "candidatos". Cada candidato generado por Autopilot consta de dos grandes pasos: el primer paso se encarga de realizar la ingeniería de features sobre el juego de datos, mientras que el segundo paso es donde se entrena un algoritmo específico para producir el modelo. Al desplegar el modelo podremos realizar inferencias siguiendo estos pasos de manera análoga.

Este notebook contiene todas las instrucciones necesarias para entrenar el modelo, así como también desplegar el mismo para realizar predicciones sobre un conjunto de datos y calcular la matriz de confusión. Usaremos el SDK de Python para AWS ([boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html)), un SDK de alto nivel que nos permitirá interactuar con Autopilot y el resto de los servicios que vamos a necesitar.

## 2. Pre-requisitos<a name="Pre-requisites"></a>

Comencemos creando los siguientes objetos en la cuenta:

- El repositorio de Amazon S3 junto con el prefijo a utilizar donde almacenaremos la información de entrenamiento y los artefactos del modelo. El mismo debería pertenecer a la misma región donde estamos corriendo el proceso de entrenamiento en SageMaker y Autopilot. El código de más abajo se encarga entonces de crear el bucket, o (si existe), usar el objeto por defecto.

- El rol de ejecución a asignarle a Autopilot para que pueda correr con un nivel de privilegios suficiente que le permita acceder a nuestros datos en S3. Para más información, visitar la documentación de Amazon SageMaker referente a roles de IAM: https://docs.aws.amazon.com/sagemaker/latest/dg/security-iam.html

In [1]:
import boto3
import sagemaker
from time import gmtime, strftime, sleep
from sagemaker import get_execution_role

region = boto3.Session().region_name

session = sagemaker.Session()
bucket = session.default_bucket()
prefix = "sagemaker/autopilot-dm"

role = get_execution_role()

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [2]:
auto_ml_job_time = strftime("%d-%H-%M-%S", gmtime())
auto_ml_job_name = "AutoPilotExperiment-{}".format(auto_ml_job_time)
print("auto_ml_job_name: " + auto_ml_job_name)

auto_ml_job_name: AutoPilotExperiment-22-01-56-30


### 2.1 Descarga de datos<a name="Downloading"></a>

Descargamos el [dataset de marketing directo](https://archive.ics.uci.edu/ml/datasets/bank+marketing) descargando una copia disponible desde el [repositorio S3 de origen](https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip). Para más información sobre esta colección de datos, visitar \[Moro et al., 2014\] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

In [3]:
!apt-get install unzip
!wget -N https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
!unzip -o bank-additional.zip

local_data_path = "./bank-additional/bank-additional-full.csv"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-23+deb10u2).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
--2022-02-22 01:56:30--  https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
Resolving sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)... 52.92.196.34
Connecting to sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)|52.92.196.34|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘bank-additional.zip’ not modified on server. Omitting download.

Archive:  bank-additional.zip
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: bank-additional/bank-additional-full.csv  


### 2.2 Carga de datos en Amazon S3<a name="Uploading"></a>

Para poder entrenar el modelo con Autopilot, necesitamos colocar los datos de entrenamiento en S3.

Primero, vamos a realizar una verificación de los datos para estar seguros de que no contenga determinados errores. En este caso, como el juego de datos es particularmente pequeño, también puede ser una opción realizar una inspección visual del mismo. Cuando los datos a procesar resultan más grandes (pudiendo incluso tener un tamaño mayor a la cantidad de memoria disponible en la instancia del bloc), se puede realizar la inspección fuera de línea usando herramientas como Apache Spark: [Deequ](https://github.com/awslabs/deequ) es un componente construido sobre Apache Spark que puede resultar útil para realizar validación de grandes colecciones de datos.

Nota: Autopilot es capaz de manejar juegos de datos de hasta 5 GB de tamaño. Para más información, visite [Cuotas de Autopilot de Amazon SageMaker](https://docs.aws.amazon.com/es_es/sagemaker/latest/dg/autopilot-quotas.html).

Realicemos una inspección visual de los datos cargando el dataset en un dataframe de Pandas en memoria.

In [4]:
import pandas as pd

data = pd.read_csv(local_data_path)
pd.set_option("display.max_columns", 500)  # Make sure we can see all of the columns
pd.set_option("display.max_rows", 10)  # Keep the output on one page
data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


Observar que tenemos 20 características o *features* para explotar en la predicción de la variable "y".

Amazon SageMaker Autopilot se encargará de realizar el pre-procesamiento por nosotros: no es necesario implementar técnicas convencionales de pre-procesamiento como por ejemplo manejo de valores faltantes, conversión de variables categóricas en numéricas, re-escalamiento o manejo de tipos de datos más complejos.

Por el mismo motivo, tampoco necesitamos particionar el conjunto de datos para entrenamiento y validación del modelo. Sin embargo, vamos a dejar una porción de los datos fuera del alcance de Autopilot para realizar una prueba del modelo más abajo.

### 2.2.1 Reserva de algunos datos para realizar la validación del modelo

Vamos a dividir los datos en 2 conjuntos mutuamente excluyentes y complementarios: entrenamiento y validación. El subconjunto de entrenamiento representa el 80% del juego de datos, y es usado para entrenar el modelo con Autopilot. En cambio, el segmento de validación (20%), será reservado para realizar inferencias de prueba sobre el modelo generado por Autopilot, más abajo.

In [5]:
train_data = data.sample(frac=0.8, random_state=200)
test_data = data.drop(train_data.index)
test_data_no_target = test_data.drop(columns=["y"])

### 2.2.2 Carga de datos en Amazon S3

Aquí copiamos los datos CSV en S3, para que puedan ser usados por los procesos de entrenamiento de modelos de SageMaker.

In [6]:
train_file = "train_data.csv"
train_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=prefix + "/train")
print("Train data uploaded to: " + train_data_s3_path)

test_file = "test_data.csv"
test_data_no_target.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(path=test_file, key_prefix=prefix + "/test")
print("Test data uploaded to: " + test_data_s3_path)

Train data uploaded to: s3://sagemaker-us-east-1-082349764434/sagemaker/autopilot-dm/train/train_data.csv
Test data uploaded to: s3://sagemaker-us-east-1-082349764434/sagemaker/autopilot-dm/test/test_data.csv


## 3. Entrenamiento del modelo con Autopilot

### 3.1 Configuración

Ahora que ya subimos los datos en Amazon S3, podemos invocar a Autopilot para entrenar el mejor modelo usando el subconjunto de datos de entrenamiento. Como vimos en el tutorial de video, las entradas requeridas para Autopilot son:

* La ubicación de los datos en S3 (datos de entrenamiento y artefactos de salida)
* El nombre de la columna a predecir (en nuestro caso es la columna "y")
* El rol de IAM, que nos permite elevar los privilegios de Autopilot para que pueda acceder a los datos alojados en S3.

En este caso los datos de entrenamiento se encuentran en formato CSV, pero Autopilot también permite entradas en formatos columnares como por ejemplo Parquet. Para más información, visite en enlace [conjuntos de datos de Autopilot de Amazon SageMaker](https://docs.aws.amazon.com/es_es/sagemaker/latest/dg/autopilot-datasets-problem-types.html#autopilot-datasets) en la [documentación de Amazon SageMaker](https://docs.aws.amazon.com/es_es/sagemaker/index.html).

*Nota: al entrenar con archivos CSV, Autopilot necesita o bien que todos los archivos posean un encabezado en la primera línea; o que el primer archivo de la colección en orden lexicográfico posea el encabezado.*

In [7]:
auto_ml_job_config = {"CompletionCriteria": {"MaxCandidates": 5}}
auto_ml_endpoint_name = ""

input_data_config = [
    {
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": "s3://{}/{}/train".format(bucket, prefix),
            }
        },
        "TargetAttributeName": "y",
    }
]

output_data_config = {"S3OutputPath": "s3://{}/{}/output".format(bucket, prefix)}

Aquí también puedes también especificar el tipo de problema a resolver: (`Regression, MulticlassClassification, BinaryClassification`). En aquellos casos en el cual esto no esté especificado (como aquí ocurre), Autopilot va a inferir el tipo de problema a partir de las estadísticas generadas sobre la columna *target* del modelo.

También tienes la opción de limintar el tiempo de corrida del ciclo de entrenamiento de Autopilot: esto puede hacerse especificando la cantidad máxima de modelos/candidatos, o bien acotando la cantidad total de tiempo del trabajo de entrenamiento. Asimismo, ten en cuenta que la cantidad de tiempo del proceso de entrenamiento puede variar entre una corrida y otra debido a la naturaleza misma del proceso. Para más información, visite la documentación del [SDK de SageMaker para Python](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_auto_ml_job).

### 3.2 Lanzamiento del proceso de entrenamiento

Para lanzar el proceso, invocamos la operación de la API `create_auto_ml_job`. 

In [8]:
sm.create_auto_ml_job(
    AutoMLJobName=auto_ml_job_name,
    InputDataConfig=input_data_config,
    OutputDataConfig=output_data_config,
    AutoMLJobConfig=auto_ml_job_config,
    RoleArn=role,
)

{'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:082349764434:automl-job/autopilotexperiment-22-01-56-30',
 'ResponseMetadata': {'RequestId': 'bf4beb36-44f4-40e6-b169-8fe7795d490a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bf4beb36-44f4-40e6-b169-8fe7795d490a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '102',
   'date': 'Tue, 22 Feb 2022 01:56:32 GMT'},
  'RetryAttempts': 0}}

### 3.3 Seguimiento del progreso del proceso de entrenamiento

A alto nivel, el proceso consta de los siguientes pasos:

* Análisis de datos. Aquí, el juego de datos de entrenamiento es usado por Autopilot para construir una lista de *pipelines* de procesamiento y diferentes algoritmos para aplicar a los datos de entrada. Asimismo, los datos son particionados en colecciones para entrenamiento y validación.
* Ingeniería de *features*, en donde Autopilot va a realizar transformaciones tanto de manera individual como así también en forma agregada.
* Ajustes del modelo, para encontrar el conjunto óptimo de hiperparámetros específicos para cada modelo.
* Identificación/selección del modelo con mejor desempeño a partir de la métrica objetivo especificada.

In [9]:
print("JobStatus - Secondary Status")
print("----------------------------")

describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

while describe_response["AutoMLJobStatus"] not in ("Failed", "Completed", "Stopped"):
    print("{} - {}".format(describe_response["AutoMLJobStatus"], describe_response["AutoMLJobSecondaryStatus"]))
    sleep(30)
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

print("{} - {}".format(describe_response["AutoMLJobStatus"], describe_response["AutoMLJobSecondaryStatus"]))

JobStatus - Secondary Status
----------------------------
InProgress - Starting
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEng

### 3.4 Cantidados explorados por SageMaker Autopilot

Puedes enumerar la lista completa de candidatos explorados Autopilot, ordenados según la métrica objetivo de performance definida en el momento de la configuración del trabajo. Aquí, entendemos por *candidato*, a una combinación de *pipeline* de procesamiento de datos, algoritmo y conjunto específico de hiperparámetros con sus valores respectivos.

In [10]:
candidates = sm.list_candidates_for_auto_ml_job(
    AutoMLJobName=auto_ml_job_name, SortBy="FinalObjectiveMetricValue"
)["Candidates"]
index = 1
for candidate in candidates:
    print(
        str(index)
        + "  "
        + candidate["CandidateName"]
        + "  "
        + str(candidate["FinalAutoMLJobObjectiveMetric"]["Value"])
    )
    index += 1

1  AutoPilotExperiment-22-01-56-3T5-001-edc1815b  0.6068300008773804
2  AutoPilotExperiment-22-01-56-3T5-002-0796d645  0.6040700078010559
3  AutoPilotExperiment-22-01-56-3T5-005-289f0c15  0.5935699939727783
4  AutoPilotExperiment-22-01-56-3T5-004-3cd99a51  0.5883200168609619
5  AutoPilotExperiment-22-01-56-3T5-003-f5a7714a  0.5880500078201294


### 3.4.1 Selección del mejor candidato

In [11]:
best_candidate = describe_response["BestCandidate"]
model_containers = best_candidate["InferenceContainers"]

print("Best candidate: {}".format(best_candidate["CandidateName"]))

Best candidate: AutoPilotExperiment-22-01-56-3T5-001-edc1815b


### 3.5 Creación del modelo

In [12]:
model_name = auto_ml_job_name
model = sm.create_model(ModelName=model_name, Containers=model_containers, ExecutionRoleArn=role)
print("Model ARN: {}".format(model["ModelArn"]))

Model ARN: arn:aws:sagemaker:us-east-1:082349764434:model/autopilotexperiment-22-01-56-30


### 3.6 *Notebook* generado para el candidato
    
Como explicamos en el tutorial de video, Sagemaker AutoPilot también genera un bloc de notas para especificar las definiciones del candidato. El mismo puede ejecutarse en forma interactiva, paso a paso para lograr una idea detallada de cómo Autopilot llega a identificar el mejor candidato; como así también para realizar todo tipo de ajustes o personalizaciones sobre el mismo: paralelismo, tipo de *hardware*, algoritmos a explorar, lógicas de extracción de *features*, etc.
    
El *notebook* puede descargarse tanto en la UI de SageMaker Studio como también de la siguiente ubicación en S3:

In [13]:
sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)["AutoMLJobArtifacts"][
    "CandidateDefinitionNotebookLocation"
]

's3://sagemaker-us-east-1-082349764434/sagemaker/autopilot-dm/output/AutoPilotExperiment-22-01-56-30/sagemaker-automl-candidates/AutoPilotExperiment-22-01-56-30-pr-1-6614cc44f08c4d9bb35f61fd52/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb'

### 3.7 *Notebook* de exploración de datos

SageMaker Autopilot también genera un bloc de notas para exploración de datos, que también está accesible tanto a través de la UI como en la siguiente ubicación de S3:

In [14]:
sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)["AutoMLJobArtifacts"][
    "DataExplorationNotebookLocation"
]

's3://sagemaker-us-east-1-082349764434/sagemaker/autopilot-dm/output/AutoPilotExperiment-22-01-56-30/sagemaker-automl-candidates/AutoPilotExperiment-22-01-56-30-pr-1-6614cc44f08c4d9bb35f61fd52/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb'

## 4 Evaluación del modelo: inferencias *online*<a name="OnlineInference"></a>

En esta sección, vamos a desplegar el modelo de mejor desempeño generado por SageMaker Autopilot en la sección 3. Para ello, vamos a tomar el conjunto de datos de validación, realizar inferencias en línea sobre el mismo, y calcular la matriz de confusión resultante.

Comencemos desplegando el *endpoint*: el mismo nos proveerá de infraestructura de cómputo administrada donde correr el modelo, junto con una API para realizar las inferencias en línea.

### 4.1 Despliegue del *endpoint* para inferencias en línea

In [15]:
auto_ml_endpoint_name = auto_ml_job_name
auto_ml_endpoint_config_name = auto_ml_endpoint_name

In [16]:
variant_name = auto_ml_job_name

production_variants = [
    {
        "InstanceType": "ml.t2.medium",
        "InitialInstanceCount": 1,
        "ModelName": model_name,
        "VariantName": variant_name
    }
]

ep_config = sm.create_endpoint_config(EndpointConfigName=auto_ml_endpoint_config_name, ProductionVariants=production_variants)
print(str(ep_config))

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:082349764434:endpoint-config/autopilotexperiment-22-01-56-30', 'ResponseMetadata': {'RequestId': '851065a6-f773-4fbb-9642-5d6bced05fb2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '851065a6-f773-4fbb-9642-5d6bced05fb2', 'content-type': 'application/x-amz-json-1.1', 'content-length': '112', 'date': 'Tue, 22 Feb 2022 02:47:15 GMT'}, 'RetryAttempts': 0}}


In [17]:
ep = sm.create_endpoint(EndpointName=auto_ml_endpoint_name, EndpointConfigName=auto_ml_endpoint_config_name)

In [18]:
ep_desc = sm.describe_endpoint(EndpointName=auto_ml_endpoint_name)

while ep_desc["EndpointStatus"] in ("Creating"):
    print("EndpointStatus: {}".format(ep_desc["EndpointStatus"]))
    sleep(30)
    ep_desc = sm.describe_endpoint(EndpointName=auto_ml_endpoint_name)

print("EndpointStatus: {}".format(ep_desc["EndpointStatus"]))

EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: Creating
EndpointStatus: InService


### 4.2 Inferencia *online* usando el subconjunto de validación

In [19]:
import numpy as np
import pandas as pd
import sagemaker

from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

In [20]:
predictor = sagemaker.predictor.Predictor(endpoint_name=auto_ml_endpoint_name,
                                          sagemaker_session=session,
                                          serializer=CSVSerializer(),
                                          deserializer=CSVDeserializer())
predictor.serializer = sagemaker.serializers.CSVSerializer()

In [21]:
predictions = predictor.predict(test_data.drop(["y"], axis=1).to_csv(sep=",", header=False, index=False))
predictions_df = pd.DataFrame(predictions)

### 4.3 Cálculo de la matriz de confusión

Calculamos la matriz de confusión. Si todo salió bien, el resultado debería promediar:
    
     predictions  0    1
         actuals
               0  6595  680
               1  160   803

In [22]:
pd.crosstab(index=pd.DataFrame(pd.get_dummies(test_data)["y_yes"].to_numpy())[0], 
            columns=pd.DataFrame(pd.get_dummies(predictions_df[0])["yes"].to_numpy())[0], 
            rownames=["actuals"], 
            colnames=["online predictions"])

online predictions,0,1
actuals,,
0,6858,417
1,323,640


### 4.4 Eliminación del endpoint de inferencia en línea

El *endpoint* de inferencia en línea posee infraestructura dedicada, teniendo [costos](https://aws.amazon.com/es/sagemaker/pricing/?nc1=h_ls) asociados que dependen del tipo de instancia, cantidad de almacenamiento y tiempo durante el cual estuvo activo. Es por esto que el mismo debe ser eliminado en cuanto ya no sea necesario.

In [23]:
# Endpoint config
desc = sm.describe_endpoint(EndpointName=auto_ml_endpoint_name)
sm.delete_endpoint_config(EndpointConfigName=desc["EndpointConfigName"])

# Endpoint
sm.delete_endpoint(EndpointName=auto_ml_endpoint_name)
print("Endpoint deleted: {}".format(auto_ml_endpoint_name))

Endpoint deleted: AutoPilotExperiment-22-01-56-30


## 5 Evaluación del modelo: inferencias *batch*<a name="BatchInference"></a>

Repitamos ahora el proceso de validación del modelo, pero usando un esquema de procesamiento fuera de línea o *batch* usando la función de [pipelines de inferencia](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html) de SageMaker. Para esto, vamos a generar un proceso de transformación apuntando a los datos de validación que tenemos en S3.

In [24]:
transform_job_name = auto_ml_job_name + auto_ml_job_time

transform_input = {
    "DataSource": {
        "S3DataSource": {
            "S3DataType": "S3Prefix", 
            "S3Uri": test_data_s3_path}
    },
    "ContentType": "text/csv",
    "CompressionType": "None",
    "SplitType": "Line",
}

transform_output = {
    "S3OutputPath": "s3://{}/{}/inference-results".format(bucket, prefix),
}

transform_resources = {
    "InstanceType": "ml.m5.large", 
    "InstanceCount": 1,
}

sm.create_transform_job(
    ModelName=model_name,
    TransformJobName=transform_job_name,
    TransformInput=transform_input,
    TransformOutput=transform_output,
    TransformResources=transform_resources,
)

{'TransformJobArn': 'arn:aws:sagemaker:us-east-1:082349764434:transform-job/autopilotexperiment-22-01-56-3022-01-56-30',
 'ResponseMetadata': {'RequestId': 'fcf4ae59-0c3d-4a91-b03c-c19ff724c769',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fcf4ae59-0c3d-4a91-b03c-c19ff724c769',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '119',
   'date': 'Tue, 22 Feb 2022 03:00:53 GMT'},
  'RetryAttempts': 0}}

In [25]:
print("Inference JobStatus")
print("-------------------")

describe_response = sm.describe_transform_job(TransformJobName=transform_job_name)

while describe_response["TransformJobStatus"] not in ("Failed", "Completed", "Stopped"):
    print(describe_response["TransformJobStatus"])
    sleep(30)
    describe_response = sm.describe_transform_job(TransformJobName=transform_job_name)

print(describe_response["TransformJobStatus"])

Inference JobStatus
-------------------
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
Completed


### 5.1 Carga y visualización de resultados

In [26]:
s3_output_key = "{}/inference-results/test_data.csv.out".format(prefix)
local_inference_results_path = "inference_results.csv"

s3 = boto3.resource("s3")
inference_results_bucket = s3.Bucket(session.default_bucket())

inference_results_bucket.download_file(s3_output_key, local_inference_results_path)

offline_df = pd.read_csv(local_inference_results_path, header=None, sep=";")
pd.set_option("display.max_rows", 10) # Mantiene la salida en una única página
offline_df

,0
0,no
1,no
2,no
3,no
4,no
...,...
8233,yes
8234,yes
8235,no
8236,yes


### 5.2 Cálculo de la matriz de confusión

En esta sección, volemos a calcular la matriz de confusión, pero esta vez usando los resultados de la inferencia fuera de línea. Si todo salió bien, el resultado debería coincidir con las predicciones en línea realizadas en la sección 4.3. Es decir, el resultado debería promediar:
    
     predictions  0    1
         actuals
               0  6595  680
               1  160   803

In [27]:
pd.crosstab(index=pd.DataFrame(pd.get_dummies(test_data["y"])["yes"].to_numpy())[0], 
            columns=pd.DataFrame(pd.get_dummies(offline_df)["0_yes"].to_numpy())[0], 
            rownames=['actuals'], 
            colnames=['offline predictions'])

offline predictions,0,1
actuals,,
0,6858,417
1,323,640


## 6. Eliminación/borrado de recursos<a name="Cleanup"></a>

### 6.1 Borrado del modelo en SageMaker

Aquí borramos el modelo que fue creado en SageMaker cuando llamamos a create_model() más arriba; es decir, no estamos borrando los artefactos del modelo, código de inferencias, o el rol asociado. Para más información, visitar la documentación de [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.delete_model).

In [28]:
# sm.delete_model(ModelName=model_name)

### 6.2 Borrado de artefactos en S3.

La corrida de Autopilot crea una cantidad de artefactos como ser por ejemplo particiones del conjunto de datos, scripts de procesamiento, etc. Activando este código, podemos eliminar estos archivos. Asimismo esta operación elimina todos los modelos y notebooks generados durante la corrida.

In [29]:
# s3 = boto3.resource("s3")
# job_outputs_bucket = s3.Bucket(bucket)
# 
# job_outputs_prefix = "{}/output/{}".format(prefix,auto_ml_job_name)
# job_outputs_bucket.objects.filter(Prefix=job_outputs_prefix).delete()